In [14]:
#get restaurant by location
import requests


headers = {
    'x-rapidapi-host': "us-restaurant-menus.p.rapidapi.com",
    'x-rapidapi-key': "your_key"
    }

querystring = {"page":"1"}


url2 ="https://us-restaurant-menus.p.rapidapi.com/restaurants/search/geo?lat=40.688072&lon=-73.997385&distance=0.3"

response_res = requests.request("GET", url2, headers=headers, params=querystring)

print(response_res.text)


{"result":{"totalResults":100,"data":[{"geo":{"lon":-73.997385,"lat":40.688072},"hours":"Mon-Thu: 11:30am-12am  Fri: 11:30am-2am  Sat: 10:30am-2am  Sun: 10:30am-12am","address":{"city":"Brooklyn","formatted":"391 Henry St Brooklyn, NY 11201","street":"391 Henry St","state":"NY","postal_code":"11201"},"restaurant_phone":"(718) 243-2522","restaurant_id":274038,"price_range":"","menus":[],"price_range_100":0,"cuisines":["Alcohol","Italian","Tapas"],"restaurant_name":"Bocca Lupo"},{"geo":{"lon":-73.997466,"lat":40.688151},"hours":"Mon-Thu: 11am-11pm  Fri-Sat: 11am-11:30am  Sun: 12pm-11pm","address":{"city":"Brooklyn","formatted":"402 Henry St Brooklyn, NY 11201","street":"402 Henry St","state":"NY","postal_code":"11201"},"restaurant_phone":"(718) 625-8583","restaurant_id":354824,"price_range":"$$","menus":[],"price_range_100":2,"cuisines":["Chinese"],"restaurant_name":"Chan's Golden City"},{"geo":{"lon":-73.997757,"lat":40.68758},"hours":"Mon-Fri: 11:30am-11pm  Sat-Sun: 12pm-11pm","address

In [15]:
#get menu items by restaurant id
url = "https://us-restaurant-menus.p.rapidapi.com/restaurant/274038/menuitems"
response_menu = requests.request("GET", url, headers=headers, params=querystring)
print(response_menu.text)

{"result":{"totalResults":180,"data":[{"address":{"city":"Brooklyn","formatted":"391 Henry St Brooklyn, NY 11201","postal_code":"11201","state":"NY","street":"391 Henry St"},"cuisines":["Alcohol","Italian","Tapas"],"geo":{"lat":40.688072,"lon":-73.997385},"item_id":1345847776,"menu_item_description":"w/ aiello's fresh ricotta & pin nuts","menu_item_name":"Roasted Beets","menu_item_pricing":[{"currency":"USD","price":10,"priceString":"$10.00"}],"price_range":"","restaurant_hours":"Mon-Thu: 11:30am-12am  Fri: 11:30am-2am  Sat: 10:30am-2am  Sun: 10:30am-12am","restaurant_id":274038,"restaurant_name":"Bocca Lupo","restaurant_phone":"(718) 243-2522","subsection":"Antipasti E Insalata","subsection_description":" "},{"address":{"city":"Brooklyn","formatted":"391 Henry St Brooklyn, NY 11201","postal_code":"11201","state":"NY","street":"391 Henry St"},"cuisines":["Alcohol","Italian","Tapas"],"geo":{"lat":40.688072,"lon":-73.997385},"item_id":1345848311,"menu_item_description":"w/ arugula","menu